# Testy statystyczne. Część I

In [16]:
import pandas as pd
from pandas import DataFrame, Series
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pingouin as pg
from pprint import pprint

## Test dwumianowy

### Przykład

Badacze przeprowadzili eksperyment, w którym prezentowali uczestnikom badania dwuznaczne zdanie "Wyprzedził autobus samochód". Zdanie to ma dwie możliwe interpretacje - a) "Autobus został wyprzedzony przez samochód" oraz b) "Samochód został wyprzedzony przez autobus". Hipoteza badawcza głosiła, że ze względu na szyk wyrazów w zdaniu to "autobus" będzie interpretowany jako podmiot zdania, wobec czego preferowaną interpretacją bęðzie interpretacja b). Aby sprawdzić tę hipotezę prezentowali 40 badanym dwuznaczne zdania i prosili o wskazanie, która interpretacja wydaje im się bardziej naturalna. Wyniki eksperymentu znajdują się w pliku `autobus.csv`. Naszym zadaniem jest sprawdzić, czy badani wybierali interpretację zakładaną przez badaczy częściej, niż gdyby wybierali losowo.

In [5]:
data = pd.read_csv('autobus.csv')
data.head()

,participant,interpretation
0,1,A
1,2,B
2,3,A
3,4,B
4,5,B


Aby sprawdzić naszą hipotezę używają testu dwumianowego, potrzebujemy najpierw zliczyć wszystkie wystąpienia interpretacji A oraz B. Posłużymy się tutaj metodą `value_counts`

In [8]:
counts = data['interpretation'].value_counts()
counts

B    26
A    14
Name: interpretation, dtype: int64

Teraz, kiedy mamy już zliczenia, możemy przeprowadzić odpowiedni test statystyczny, posługując się metodą `binom_test` z modułu `scipy.stats`. Funkcja ta zwraca wartość $p$ dla testu. Argument `p` służy do przekazania, jakie prawdopodobieństwo sukcesu zakłada hipoteza zerowa. W naszym przypadku jest to $0.5$.

In [9]:
stats.binom_test(counts, p=0.5)

0.08069046775199244

Okazuje się, że przy poziomie istotności statystycznej $\alpha = 0.05$ nie mamy podstaw, aby odrzucić hipotezę zerową i przyjąć hipotezę alternatywną.

Uwaga! Funkcja ta przeprowadza test dwustronny. Dla chętnych i zainteresowanych - jak moglibyśmy przeprowadzić test jednostronny w Pythonie?

## Test zgodności $\chi^2$

### Przykład

Ci sami badacze postanowili przeprowadzić badania dotyczące interpretacji kwantyfikatorów w języku polskim w tzw. "donkey sentences" (oślich zdaniach). Ich zdaniem w przypadku zdań w rodzaju "Każdy farmer, który ma osła, bije go" możliwe są trzy interpretacje:
* A: 
* B:
* C:

Hipoteza badaczy głosiła, że nie ma jednej preferowanej interpretacji tego rodzaju zdań. Zamiast tego, uważali naukowcy, w populacji kompetentnych użytkowników języka polskiego istnieje 1/3 osób, które ośle zdania interpretują na sposób A, 1/3, które interpretują na sposób B oraz 1/3, które interpretują je jako C. Badacze przeprowadzili eksperyment, w którym okazało się, że faktycznie sposób interpretacji jest kwestią osobniczą - badani byli bardzo spójni w swoich interpretacjach. Do przetestowania pozostał tylko rozkład różnych sposobów interpretacji w populacji. Musimy sprawdzić, czy dane wspierają hipotezę, zgodnie z którą warianty A, B oraz C rozkładają się po równo. Wyniki eksperymentu przeprowadzonego na 70 osobach znajdują się w pliku `osly.csv`.

In [25]:
data = pd.read_csv('osly.csv')
data.head(8)

,participant,donkey_interp
0,1,A
1,2,A
2,3,B
3,4,A
4,5,B
5,6,C
6,7,C
7,8,A


Podobnie jak w poprzednim przypadku do sprawdzenia hipotezy musimy najpierw obliczyć jak dużo mamy obserwacji, w których badani interpretowali ośle zdania w określony sposób. Znów posłużymy się metodą `value_counts`. 

In [32]:
counts = data['donkey_interp'].value_counts()
counts

C    25
B    25
A    20
Name: donkey_interp, dtype: int64

Test zgodności $\chi^2$ możemy przeprowadzić za pomocą funkcji `chisqare` z modułu `scipy.stats`. Funkcja ta przyjmuje dwa argumenty - jednym z nich są zliczenia, drugim oczekiwane wartości. W naszym wypadku (mamy 70 obserwacji) oczekujemy, że w każdej kategorii (A, B oraz C) będziemy mieli 70/3 zliczeń.

In [31]:
chi2, p = stats.chisquare(counts,
                          [70/3, 70/3, 70/3])
print('Statystyka testowa chi2: ', chi2)
print('Wartość p: ', p)

Statystyka testowa chi2:  0.7142857142857143
Wartość p:  0.6996725373751302


Okazuje się, że na podstawie testu zgodności $\chi^2$ nie możemy odrzucić hipotezy zerowej. Do pewnego stopnia wspiera to hipotezy badaczy (ale należy pamiętać, że brak odrzucenia hipotezy zerowej nie oznacza jej potwierdzenia!)

## Test niezależności $\chi^2$

### Przykład

Badacze postanowili kontynuować swoje dociekania nad interpretacją dwuznacznych zdań w języku polskim z pierwszego przykładu. Uznali, że skoro według ich hipotezy badawczej wpływ na preferencje interpretacyjne takich zdań ma kolejność słów w zdaniu, to należy podzielić badanych na dwie grupy. Jedna z nich ($N=50$) (grupa A-S) otrzymywała do oceny zdanie "Wyprzedził autobus samochód", druga zaś ($N=45$) zdanie "Wyprzedził samochód autobus" (grupa S-A). Zadanie eksperymentalne było identyczne - badani musieli zdecydować, która interpretacja wydaje im się bardziej naturalna. Skoro hipoteza badawcza głosi więc, że kolejność słów w zdaniu ma znaczenie dla preferencji interpretacyjnej, to hipoteza zerowa głosić będzie, że te dwie zmienne są niezależne. Wyniki eksperymentu znajdują się w pliku `autobus2.csv`. Naszym zadaniem jest sprawdzić, czy dane wspierają hipotezę postawioną przez badaczy.

In [11]:
data = pd.read_csv('autobus2.csv')
data.head()

,participant,condition,interpretation
0,1,A-S,B
1,2,A-S,B
2,3,A-S,B
3,4,A-S,A
4,5,A-S,B


Do przeprowadzenia testu niezależności $\chi^2$ potrzebujemy wykonać tabelę krzyżową. Całe szczęście z pomocą przychodzi nam funkcja `crosstab` z pakietu `pandas`. Dzięki niej łatwo możemy zobaczyć rozkład odpowiedzi badanych w obu warunkach eksperymentalnych:

In [14]:
xtable = pd.crosstab(data['interpretation'], data['condition'])
xtable

condition,A-S,S-A
interpretation,,
A,18,31
B,32,14


Następnie za pomocą funkcji `chi2_contingency` z modułu `scipy.stats` przeprowadzimy test niezależności $\chi^2$. Funkcja ta przyjmuje tabelę z danymi RXC (może to być ramka danych, może to również być obiekt klasy `np.array`). Jako rezultat zwraca 4 wartości - wartość statystyki testowej, wartość p, liczbę stopni swobody oraz tablicę z wartościami oczekiwanymi użytymi do kalkulacji.

In [22]:
chi2, p, dof, expected = stats.chi2_contingency(xtable)
print('Statystyka testowa chi2: ', chi2)
print('Wartość p: ', p)
print('Liczba stopni swobody: ', dof)
print('Wartości oczekiwane: \n', expected)

Statystyka testowa chi2:  8.983110273094749
Wartość p:  0.0027248644001806313
Liczba stopni swobody:  1
Wartości oczekiwane: 
 [[25.78947368 23.21052632]
 [24.21052632 21.78947368]]


## Test t Studenta dla jednej próby

### Przykład

Grupa badaczy testowała interpretację konstrukcji "jeżeli.. to..." w języku naturalnym. Hipoteza badawcza głosiła, że jeżeli następnik znajduje się jako ostatni w zdaniu, to kompetentni użytkownicy języka interpretują taką konstrukcję jako implikację materialną, jeżeli zaś jako pierwszy, to nadają takiemu zdaniu silniejszą interpretację i między poprzednikiem a następnikiem musi zachodzić relacja wynikania. W celu sprawdzenia swojej hipotezy badacze przedstawili uczestnikom ($N=126$) 2 zdania i prosili o ocenę ich akceptowalności na 9-stopniowej skali Likerta:
* LAST: Jeżeli Warszawa jest stolicą Węgier, to Księżyc jest z sera.
* FIST: Księżyc jest z sera, jeżeli Warszawa jest stolicą Węgier.

Hipoteza badawcza głosiła, że zdanie LAST będzie nieakceptowalne dla badanych (średnia poniżej 5) podczas gdy zdanie FIST będzie akceptowalne (średnia powyżej 5). Wyniki eksperymentu znajdują się w pliku `sentence_types.csv`. Naszym zadaniem jest zobaczenie, czy hipotezy badawcze są wsparte przez zebrane przez naukowców dane empiryczne. 

In [117]:
data = pd.read_csv('sentence.types.csv')
data.head()

,Unnamed: 0,participant,sentence_type,sentence_accept
0,0,1,last,3
1,1,1,first,4
2,0,2,last,4
3,1,2,first,1
4,0,3,last,2


Najpierw podzielimy nasz zbiór danych na obserwacje, w których oceniane było zdanie typu LAST oraz takie, w których oceniane było zdanie typu FIRST.

In [118]:
type_last = data[data['sentence_type'] == 'last']
type_last.head()

,Unnamed: 0,participant,sentence_type,sentence_accept
0,0,1,last,3
2,0,2,last,4
4,0,3,last,2
6,0,4,last,3
8,0,5,last,7


In [124]:
type_first = data[data['sentence_type'] == 'first']
type_first.head()

,Unnamed: 0,participant,sentence_type,sentence_accept
1,1,1,first,4
3,1,2,first,1
5,1,3,first,3
7,1,4,first,6
9,1,5,first,4


Następnie obliczyć możemy średnią akceptację dla tych zdań. W obu przypadkach jest to około 4.3.

In [119]:
type_last['sentence_accept'].mean()

4.246031746031746

In [126]:
type_first['sentence_accept'].mean()

4.349206349206349

Nasza hipoteza badawcza głosiła, że zdania typu LAST nie będą akceptowane podczas gdy zdania typu FIRST będą. Okazuje się, że jeśli już, to oba rodzaje zdań nie są akceptowane. Chcemy jednak sprawdzić, czy uzyskane wyniki w sposób statystycznie istotny różnią się od wartości stanowiącej środek skali (0.5). W tym celu posłużymy się testem t Studenta dla jednej próby.

Pierwszy sposób na wykonanie tego testu to użycie funkcji `ttest_1samp` z modułu `scipy.stats`. Funkcja ta przyjmuje jako swój argument listę (bądź `Series`) z obserwacjami. Dodatkowo możemy określić średnią populacji, dla której przeprowadzamy test za pomocą argumentu `popmean`. U nas, zgodnie z hipotezą badawczą, jest to $5$.

In [140]:
t, p = stats.ttest_1samp(type_last['sentence_accept'], popmean=5)
print('Test t Studenta dla zdania typu LAST')
print('Statystyka testowa T: ', t)
print('Wartość p: ', p)

Test t Studenta dla zdania typu LAST
Statystyka testowa T:  -5.6409482620398395
Wartość p:  1.0731493720922533e-07


In [141]:
t, p = stats.ttest_1samp(type_first['sentence_accept'], popmean=5)
print('Test t Studenta dla zdania typu FIRST')
print('Statystyka testowa T: ', t)
print('Wartość p: ', p)

Test t Studenta dla zdania typu FIRST
Statystyka testowa T:  -5.020650732842313
Wartość p:  1.7313251337168195e-06


Okazuje się, że w obu przypadkach mamy podstawy do odrzucenia hipotezy zerowej. Zarówno zdania typu LAST jak i FIRST nie są akceptowalne przez kompetentnych użytkowników polskiego.

Alternatywnym (i bardziej efektownym) sposobem jest skorzystanie z funkcji `ttest` zawartej w pakiecie `pingouin`. Oprócz tego, że zwraca wyniki w ładniejszym formacie, to dodatkowo otrzymujemy inne przydatne informacje (np. wielkość efektu w kolumnie `cohen-d`).

In [143]:
pg.ttest(type_last['sentence_accept'], 5)

,T,p-val,dof,tail,cohen-d,power,BF10
T-test,-5.641,1.073149e-07,125,two-sided,0.503,1.0,107330.417


## Test t Studenta dla prób zależnych

### Przykład

Grupa badaczy postanowiła sprawdzić, czy w sytuacji, w której jesteśmu uczuleni na możliwość pomyłki, standard oceny prawdziwości zdań, przypisujących wiedzę może się zmienić. W tym celu zaprezentowali badanym ($N=46$) krótką historyjkę:

> Maria spytała Jacka: "Jacku, czy wiesz która jest godzina?". Jacek spojrzał na zegarek i odrzekł "Mario, jest godzina 14:20".

Następnie poprosili badanych o wyrażenie zgody na zdanie:

> Jacek wie, że jest godzina 14:20

Po udzieleniu odpowiedzi na to pytanie badanym prezentowany był dodatkowy fragment historyjki:

> Jacek bardzo rzadko korzysta z zegarka i nie sprawdzał bardzo dawno czy się nie spieszy czy się nie spóźnia. Dodatkowo należy zauważyć, że patrzył na zegarek tylko pół sekundy i nie sprawdził wcześniej, czy nie ma go założonego odwrotnie.

Po przedstawieniu tego fragmentu badacze ponownie prosili uczestników o wyrażenie zgody na 9-stopniowej skali Likerta na zdanie:

> Jacek wie, że jest godzina 14:20

Hipoteza badawcza głosiła, że po uczuleniu na możliwość pomyłki akceptacja dla atrybucji wiedzy będzie niższa niż przed. Czy dane wspierają hipotezy badaczy? 

In [144]:
data = pd.read_csv('ratings.csv')
data.head()

,Unnamed: 0,participant,rating1,rating2
0,0,1,5,2
1,0,2,6,2
2,0,3,4,5
3,0,4,6,5
4,0,5,8,4


Aby sprawdzić hipotezę badaczy wystarczy zastosować test t Studenta dla prób zależnych (bo badanie zostało przeprowadzone w schemacie wewnątrzgrupowym). Możemy to zrobić używając funkcji `ttest_rel` z modułu `scipy.stats`. Przyjmuje ona jako swoje argumenty dwie listy z obserwacjami (uwaga! sami musimy zadbać, żeby były odpowiednio ułożone).

In [132]:
t, p = stats.ttest_rel(data['rating1'], data['rating2'])
print('Statystyka testowa T: ', t)
print('Wartość p: ', p)

Statystyka testowa T:  7.077350414448114
Wartość p:  6.976763876158575e-09


To samo możemy zrobić przy użyciu pakietu `pingouin`. Zawarta tam funkcja `ttest` przyjmuje argument `paired` pozwalający przekazać, że chodzi nam o próby zależne.

In [133]:
pg.ttest(data['rating1'], data['rating2'], paired=True)

,T,p-val,dof,tail,cohen-d,power,BF10
T-test,7.077,6.976764e-09,46,two-sided,1.321,1.0,1781649.024


Okazało się, że mamy prawo odrzucić hipotezę zerową i przyjąć hipotezę alternatywną.

## Test t Studenta dla prób niezależnych

### Przykład

Wielu filozofów języka uważa, że zdania z pustymi deskrypcjami określonymi, takie jak "Obecny król Francji jest łysy", są fałszywe. Inni zaś uważają, że raczej są pozbawione wartości logicznej. Nasi badacze postawili jednak hipotezę, że zależne jest to od wykształcenia logicznego. Aby sprawdzić tę hipotezę przeprowadzili badanie na grupie 50 studentów kognitywistyki przez kursem logiki oraz 45 studentów kulturoznawstwa przed kursem logiki. Badani w obu grupach mieli w kwestionariuszu ocenić akceptowalność 10 zdań w rodzaju "Obecny król Francji jest łysy". Oprócz pozycji eksperymentalnych w kwestionariuszu znalazły się również 10 "wypełniacze". Dane z eksperymentu znajdują się w pliku `acceptability.csv`. Naszym zadaniem jest sprawdzić, czy wspierają hipotezę o wpływie kursu logiki na ewaluację zdań z pustymi deskrypcjami określonymi. 

In [190]:
data = pd.read_csv('acceptability.csv')
data.head()

,Unnamed: 0,participant,condition,item,response
0,0,1,experimental,target1,3
1,1,1,experimental,target2,1
2,2,1,experimental,target3,5
3,3,1,experimental,target4,6
4,4,1,experimental,target5,6


Na początek musimy wykonać dwie operacje. Pierwszą jest usunięcie ze zbioru danych tych obserwacji, które dotyczą "wypełniaczy". Po drugie chcemy podzielić nasze dane na dwa zbiory - jeden dotyczący grupy eksperymentalnej, drugi kontrolnej.

In [191]:
data = data[data['item'].str.startswith('target')] # przydatna funkcja
experimental = data[data['condition'] == 'experimental']
control = data[data['condition'] == 'control']

Następnie chcielibyśmy obliczyć średnią z odpowiedzi na pozycje eksperymentalne dla każdego badanego. Posłużymy się tutaj funkcją `groupby`.

In [192]:
gb = experimental.groupby('participant')
exp_response = gb['response'].mean()
exp_response.head()

participant
1    3.8
2    3.1
3    4.0
4    3.5
5    3.5
Name: response, dtype: float64

In [193]:
gb = control.groupby('participant')
contr_response = gb['response'].mean()
contr_response.head()

participant
26    3.6
27    3.9
28    4.5
29    4.2
30    4.9
Name: response, dtype: float64

In [194]:
print('Średnia dla grupy eksperymentalnej: ', exp_response.mean())
print('Średnia dla grupy kontrolnej: ', contr_response.mean())


Średnia dla grupy eksperymentalnej:  3.6839999999999997
Średnia dla grupy kontrolnej:  4.2


Teraz, kiedy nasze dane są już odpowiednio zagregowane, możemy przeprowadzić test t Studenta dla prób niezależnych. Aby to zrobić możemy wykorzystać funkcję `ttest_ind` z modułu `scipy.stats`. Przyjmuje ona dwie listy z obserwacjami i zwraca statystykę testową ($t$) oraz wartość $p$.

In [195]:
t, p = stats.ttest_ind(exp_response, contr_response)
print('Statystyka testowa T: ', t)
print('Wartość p: ', p)

Statystyka testowa T:  -3.9862499406286753
Wartość p:  0.00022769869649598898


Naturalnie jeżeli zależy nam na ładnym wyglądzie i dodatkowych informacjach możemy skorzystać z `ttest` z pakietu `pingouin`.

In [196]:
pg.ttest(contr_response, exp_response)

,T,p-val,dof,tail,cohen-d,power,BF10
T-test,3.986,0.000228,48,two-sided,1.127,0.974,108.482


## Wielkość efektu i analiza mocy dla testu t Studenta

### Przykład

Załóżmy, że chcielibyśmy zreplikować to badanie. Uważamy, że był w nim przynajmniej jeden czynnik zakłócający - studenci z grupy kontrolnej i eksperymentalnej byli z innych kierunków. My chcemy przeprowadzić to badanie na studentach kognitywistyki przed i po kursie logiki.

Chcielibyśmy aby nasza replikacja miała odpowiednią moc. Załóżmy, że chcemy osiągnąć moc rzędu 0.9 (to znaczy mieć 80% szans na wykrycie efektu o określonej wielkości lub większej). Jak dużej próby potrzebujemy? Załóżmy, że chcemy przyjąć poziom istotności $\alpha = 0.01$. Wiemy, że w oryginalnym badaniu wielkość efektu (wyrażona w d Cohena) wynosiła $d=1.127$. Aby obliczyć odpowiednie wartości użyjemy funkcji `power_ttest` z pakietu `pingouin`. Jest to reimplementacja odpowiedniej funkcji z pakietu `pwr` w R.

In [212]:
# Uwaga! Aby funkcja działała musimy pominąć dokładnie 1 z parametrów: `d`, `n`, `power` i `alpha`
n = pg.power_ttest(d = 1.127, 
                   power=0.9, 
                   contrast='two-samples', # to jest domyślna wartość w tej funkcji
                   tail='two-sided', # to jest domyślna wartość w tej funkcji
                   alpha = 0.01) # domyślnie jest 0.05
n

25.144762136538294

Aby móc zreplikować to badanie przy założonej mocy i istotności statystycznej potrzebujemy około 25 osób na grupę.

Niektórzy proponują, aby w badaniach replikacyjnych założyć, że chcemy mieć 80% prawdopodobieństwa na wykrycie efektu, na wykrycie którego oryginalny eksperyment miał 30%. Przeanalizujmy tę sytuację.

In [208]:
d = pg.power_ttest2n(power=0.30, alpha=0.01, nx=50, ny=45)
d

0.4291617531866683

Dowiedzieliśmy się, że oryginalny eksperyment miał 30% szans na wykrycie efektu o wielkości $d = 0.43$. Chcemy, aby nasza replikacja miała odpowiednio większą moc (0.8 w tym przypadku).

In [210]:
pg.power_ttest(d = d, power = 0.8, alpha = 0.01)

128.4898300229837

W takim wypadku potrzebujemy 129 osób w każdej grupie w eksperymencie! To może być trudne!